In [1]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.callbacks import History, ModelCheckpoint
from sklearn.model_selection import train_test_split
from ssd300 import build_SSD300
from parser import Parser
import os
from Decoder import Decoder
from Encoder import Encoder

Using TensorFlow backend.


In [2]:
# Configuration of the model: 
input_shape=(300, 300, 3)
numClasses = 10
iou_thres=0.5 # for default and gt matching
nms_thres=0.45 # IoU threshold for non-maximal suppression
score_thres=0.01 # threshold for classification scores
top_k=200 # the maximum number of predictions kept per image
min_scale=0.2 # the smallest scale of the feature map
max_scale=0.9 # the largest scale of the feature map
aspect_ratios=[0.5, 1, 2] # aspect ratios of the default boxes to be generated
n_predictions=6 # the number of prediction blocks
prediction_size=[37, 18, 10, 5, 3, 1] # sizes of feature maps at each level

### Parse Input: 


In [3]:
data_dir = "/Users/tranle/mscoco"
data_type = "val2017"

# Initialize a parser object
parser = Parser(data_dir, data_type)

# Load images and annotations for the image
# For now, we load only 10 first classes and images are resize to (300,300,3) for training purposes

X, Y = parser.load_data()

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


### Ecode Parsed Input: 
In this step, we perform bipartite matching and multi-matching algorithm to match default boxes generated with the ground-truth labels of our training data. 

In [4]:
# Initialize an Ecoder object
encode = Encoder(y_truth=Y, 
                numClasses=numClasses, 
                iou_thres=iou_thres,
                min_scale=min_scale, 
                max_scale=max_scale, 
                aspect_ratios=aspect_ratios, 
                n_predictions=n_predictions, 
                prediction_size=prediction_size)

# Encode the labels and ground-truth boxes of the training images
Y = encode.get_encoded_data()

# Split the data into train and test datasets
X_train, Y_train, X_test, Y_test = train_test_split(X, Y, test_size=0.2)

NameError: name 'aspect_ratio' is not defined

In [ ]:
# Build the SSD model
K.clear_session() # Clear previous session

# Build the model
model = build_SSD300(input_shape=input_shape, 
                  numClasses=numClasses, 
                  mode='training', 
                  min_scale=min_scale, 
                  max_scale=max_scale, 
                  aspect_ratios=aspect_ratios, 
                  iou_thres=iou_thres,
                  nms_thres=nms_thres, 
                  score_thres=score_thres, 
                  top_k=top_k,
                  n_predictions=n_predictions)

# Instantiate the Adam optimizer for the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compile the model
model.compile(optimizer=adam, loss=loss_function, 
             metrics=['accuracy'])



### Create checkpoints to store learned weights: 


In [ ]:
# Path to store learn weights
path = os.getcwd()

checkpoints = ModelCheckpoint(filepath=ssd7_epoch-{epoch:2d}_loss-{loss:.4f}_val_loss
                              -{val_loss:.4f}.h5', 
                             monitor='val_loss'
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='min', 
                             period=1)